# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

## Trade Analytics

In the previous notebook, you learned how to define a strategy's trading conditions and generate signals over a historical time period.

In this notebook, you will learn how to evaluate the performance of your trading strategy. Simply looking at the overall profit or loss is not the most effective way to analyse a trading strategy. To assess if a strategy is viable, we must also analyse factors such as time taken, number of trades, and average profit or loss on each trade.
The key steps are:
1. [Read the Data](#read)
2. [Different Trade Level Analytics](#trade)<br>
   2.1. [Profit and Loss](#pnl)<br>
   2.2. [Win Percentage](#win)<br>
   2.3. [Average Profit Per Trade](#avg)<br>
   2.4. [Average Trade Duration](#time)<br>
   2.5. [Profit Factor](#profit)<br>

In [1]:
# For Data manipulation
import pandas as pd
import numpy as np

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

<a id='read'></a>
## Read the Data 

We have saved the trades data in a CSV file named `marubozu_trades.csv`. You can read the file using the `pandas.read_csv()` method. 

In [2]:
# Read the data
trades = pd.read_csv('../data_modules/marubozu_trades.csv', index_col=0)

trades.head()

,Entry_Date,Entry_Price,Exit_Date,Exit_Type,Exit_Price,PnL
0,2017-10-06,38.742500,2017-10-16,TP,39.970001,1.23
1,2018-01-11,43.647499,2018-01-12,TP,44.272499,0.62
2,2018-02-15,42.447498,2018-05-04,TP,45.957500,3.51
3,2018-05-11,47.372501,2018-06-25,SL,45.542500,-1.83
4,2018-09-25,54.937500,2018-09-27,TP,56.237499,1.30


<a id='trade'></a>
## Different Trade Level Analytics

In this section, we will look at some different trade level analytics. 

First, we will create a dataframe named `analytics` to store all the metrics we calculate in this notebook. 

In [3]:
# Create dataframe to store trade analytics
analytics = pd.DataFrame(index=['Strategy'])

<a id='pnl'></a>
### Profit and Loss 
The profit and loss metric is nothing but the sum of all the gains and losses that were incurred on all the trades. 

We will calculate the PnL using the `sum()` function on the column named `PnL` in our dataset. This will tell us if our strategy incurred an overall profit or loss. 

In [4]:
# Calculate total PnL
analytics['Total PnL'] = trades.PnL.sum() 

# Print the value
print("Total PnL: ",analytics['Total PnL'][0])

Total PnL:  57.73


The value returned is positive, which indicates that the strategy earned a total profit of around $57.73. However, we cannot measure a strategy's success just on the basis of `Pnl`, as it provides no information regarding the number of trades and win percentage to better evaluate the strategy.

<a id='win'></a>
### Win Percentage
The win percentage or win rate is an essential metric. It represents the percentage of trades which were profitable out of the total trades, to determine a strategy's success. A win rate above 50% is usually favourable.

Calculating the win percentage takes the number of profitable trades divided by the total number of trades as shown below. 

$$ Win~Rate = \frac{No.~of~Winning~Trades}{Total~No.~of~Trades} *100$$

Similarly, we can also calculate the Loss Percentage by taking the number of losing trades divided by the total number of trades. First, let's find the total number of long and short trades we have made from the strategy. 

In [5]:
# Number of total trades
analytics['total_trades'] = len(trades)

Next, we will find the number of winners and losers. Trades that gave us a profit i.e. `Pnl`>0 are winning trades. Trades that gave us a loss or no profit i.e. `Pnl`<=0 are losing trades. 

In [6]:
# Profitable trades
analytics['Number of Winners'] = len(trades.loc[trades.PnL>0])

# Loss-making trades
analytics['Number of Losers'] = len(trades.loc[trades.PnL<=0])

Now applying the formula discussed earlier, we can calculate the win and loss percentages, respectively. 

In [7]:
# Win percentage
analytics['Win (%)'] = 100*analytics['Number of Winners']/analytics.total_trades

# Loss percentage
analytics['Loss (%)'] = 100*analytics['Number of Losers']/analytics.total_trades

analytics.T

,Strategy
Total PnL,57.730000
total_trades,33.000000
Number of Winners,25.000000
Number of Losers,8.000000
Win (%),75.757576
Loss (%),24.242424


The win percentage comes out to be 75.75%. This indicates that we have a considerable number of trades which were profitable. But around 24.24% of total trades were unsuccessful. <br>
<b>Does this mean our strategy did not perform well?</b><br>

Not necessarily. A win rate around 50% or even below 50% does not always imply that our strategy failed to make money. 
It may sound logical that you can only make money if you have more winners than losers, but this is not always true. If your winners are giving much higher returns than your losers then you can still make good profits with a relatively low win rate. 

<a id='avg'></a>
### Average PnL Per Trade
The average PnL per trade is used to find the average amount that you can expect to gain or lose on each trade. This metric tells us how much impact a winning or losing trade might have. In general, we want the average loss per losing trade to be as low as possible and the average profit per winning trade as high as possible. For example, if your average loss per losing trade is 3x your average profit per winning trade then that means that a single loser will wipe out the profits of 3 winners.

You can determine the average profit per winning trade by dividing the sum amount of all the profit by the number of winning trades.

$$ Average~Profit~Per~Winning~Trade = \frac{Total~Profit~made~by~all~Winners}{No.~of~Winning~Trades} $$

Similarly, you can find average loss per losing trade by dividing the sum of all the losses by the number of losing trades.


In [8]:
# Per trade profit/loss of winning trades
analytics['per_trade_PnL_winners'] = trades.loc[trades.PnL>0].PnL.mean()

# Per trade profit/loss of losing trades
analytics['per_trade_PnL_losers'] = np.abs(trades.loc[trades.PnL<=0].PnL.mean())

analytics.T

,Strategy
Total PnL,57.730000
total_trades,33.000000
Number of Winners,25.000000
Number of Losers,8.000000
Win (%),75.757576
Loss (%),24.242424
per_trade_PnL_winners,4.466800
per_trade_PnL_losers,6.742500


<a id='time'></a>
### Average Trade Duration
The average trade duration, also known as the average holding period, is the amount of time you remain in a trade on average. This is important because your capital is ‘locked’ during the time of the trade and cannot be used for other trades. That way it limits the number of trades you can take simultaneously. Hence, limiting your potential to increase profits. Another risk of a long holding period is the release of important news or earning reports. This can violently change the market and may result in a losing trade. We will learn to handle such changes in the risk management section later in this course. 

Conversely, a short holding period is also not favourable, as it results in higher transaction costs and may eat away at your profits. 

To calculate the average trade duration we first calculate the holding period per trade i.e. `Exit Date` - `Entry Date`. Next, we calculate the mean of the holding time using the `mean()` method. 

In [9]:
# Convert entry time and exit time to datetime format
trades['Entry Date'] = pd.to_datetime(trades['Entry_Date'])
trades['Exit Date'] = pd.to_datetime(trades['Exit_Date'])

# Calculate holding period for each trade
holding_period = trades['Exit Date'] - trades['Entry Date']

# Calculate their mean
analytics['Average holding time'] = holding_period.mean()

analytics.T

,Strategy
Total PnL,57.73
total_trades,33
Number of Winners,25
Number of Losers,8
Win (%),75.757576
Loss (%),24.242424
per_trade_PnL_winners,4.4668
per_trade_PnL_losers,6.7425
Average holding time,13 days 22:32:43.636363636


We get the average holding period as 13 days and 22 hours which is approximately 14 days. This is neither too large for sudden market changes nor too small for high transaction costs. 
<a id='profit'></a>
### Profit Factor
The profit factor measures the amount of money made against the money lost while trading. 
It is the ratio of the sum of profit to the sum of loss. It can also be calculated with following formula: 

$$ Profit~Factor = \frac{~Win~Percentage~*~Average~Profit~Per~Winning~Trade}{~Loss~Percentage~*~Average~Loss~Per~Losing~Trade} $$

Ideally a profit factor greater than 1 is desired. Anything below one is considered as unsatisfactory performance. There is a grading system for the profit factor to help you analyse the performance of your strategy. 

|S.No | Profit Factor | Interpretation    |
|---:|:-------------|:-----------|
| 1 | Below 1  | Strategy is unprofitable |
| 2 | Equal to 1  | Capital at the time of exit is same as capital at time of entry | 
| 3 | Between 1.10 and 1.40 | Strategy provides average returns, but may not withstand high volatility | 
| 4 | Between 1.40 and 2.0 | Strategy is decent | 
| 5 | Equal to or greater than 2  | Strategy is excellent | 

In [10]:
# Calculate profit factor
analytics['Profit Factor'] = (analytics['Win (%)']/100*analytics['per_trade_PnL_winners'])/(analytics['Loss (%)']/100*analytics['per_trade_PnL_losers'])

analytics.T

,Strategy
Total PnL,57.73
total_trades,33
Number of Winners,25
Number of Losers,8
Win (%),75.757576
Loss (%),24.242424
per_trade_PnL_winners,4.4668
per_trade_PnL_losers,6.7425
Average holding time,13 days 22:32:43.636363636
Profit Factor,2.070263


Thus the profit factor comes out to be `2.07`. This means our strategy has gained `2.07` dollars for every lost dollar.

## Conclusion
In this notebook, we learned how to use a few trading metrics to analyse our trading strategy. These analytics help you find how the strategy has performed after the trade has been executed. However, it is also important to measure how the strategy has performed within the trade. We will evaluate this in the next notebook with the help of some performance metrics. <br><br>